# Introduction
This project investigates the factors of happiness. We use (source)'s definiton of happiness.
We hypothesized on following factors: economic well being (in terms of GDP per capita), education (in terms of spending), internet propogation (area of coverage in cities), life expectency and other social factors.\ 
We aim to answer what is the relationship between these factors in countries and citizens' happiness? and why do some countries report high levels of social well-being despite having a lower GDP?\
\
Our team consists of:
* Aslbek: Data processor
* Kassymkhan: Analyzer
* Olzhas: Clusterer
* Yernur: Plotter
\
We divided our research into three parts:
**Data processing** - to ensure that we have meaning and don't loose too much data\
**Data analysis** - We used various data visualization techniques and machine learning models in order to have insights\
**Findings and Evaluation.** Finally, using prompt engineering, we synthesized findings and generated AI-assisted policy insights.\
\
Prerequsites are put in the requirements.txt file.\
Our used data can be found in the data directory. It has raw/ and processed/ subdirectories which contain corresponding data.\
The dashboard is available on the dash_board.py file. In order to use it, just run it. It will open the dashboard in the browser.\
The non-code version of this file with all the charts can be found in the "report.pdf" file (credit for writing the report: Kassymkhan).\

# Data processing

In [1]:
# General
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Data cleaning with KNN
from sklearn.impute import KNNImputer

# Linear regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Polynomial regression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# Clusters
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Dashboard
import plotly.express as px

## Data Preprocessing
In order to keep the focus, we narrowed down our scope to statistics of year 2019.

We used the follwing data:
* Happiness data set (source: ) - this is a data from . Further, 'hap' for short;
* GDP per capita (source: world bank) - this is a data from . Further, 'gdp' for short;
* Education expenditure (source: ) - this is a data from . Further, 'edu' for short;
* Life expectancy (source: ) - this is a data from . Further, 'exp' for short;
* Inter penetrations (source: ) - this is a data from . Further, 'pen' for short.
* 

After reading the files, we can immediately notice different number of rows in each data frame:
* hap - 156 rows;
* gdp - 266 rows;
* hap - 266 rows;
*

This is because World Bank includes names of sets of regions, e.g. 'European Union', 'Caribbean Islands', 'World'.\
We will use left-merge to extend the hap data frame, as it has the most relevant name for regions.\
But first, we must prepare our data. We begin by introducing common column labels.

In [4]:
# ================================
# Happiness Dataset
# ================================
hap_raw = pd.read_csv("./data/raw/happiness.csv")

hap = hap_raw.rename(columns={
    "Country or region": "Country",
    "Score": "Happiness score",
    "GDP per capita": "Happiness GDP",
})

hap["Year"] = 2019
hap = hap.drop(columns=["Year", "Overall rank"])

print("Happiness statistics:")
hap


FileNotFoundError: [Errno 2] No such file or directory: './data/raw/happiness.csv'

In [ ]:
# ================================
# GDP per capita
# ================================
gdp_raw = pd.read_excel("GDP.xls")

gdp = gdp_raw[["Country Name", "2019"]]

gdp = gdp.rename(columns={
    "Country Name": "Country",
    "2019": "GDP per capita"
})

print("GDP per capita")
gdp


In [ ]:
# ================================
# Education Expenditure
# ================================
edu_raw = pd.read_excel("Education.xls")

edu = edu_raw[["Country Name", "2019"]]

edu = edu.rename(columns={
    "Country Name": "Country",
    "2019": "Education expenditure"
})

print("Education expenditure:")
edu


In [ ]:
# ================================
# Life Expectancy Rates
# ================================
exp_raw=pd.read_excel("LifeExpectancy.xls", header=3)
id_vars = ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code']
exp_melted = exp_raw.melt(
    id_vars=id_vars, 
    var_name='Year', 
    value_name='Life expectancy' # Set the value column name directly
)

exp_2019 = exp_melted[exp_melted['Year'] == '2019'].copy()
exp_2019 = exp_2019.rename(columns={'Country Name': 'Country'})
exp = exp_2019[['Country', 'Life expectancy']]
exp


In [ ]:
# ================================
# Internet Penetration Rates
# ================================
net_raw = pd.read_excel("InternetRates.xls", header=3)
id_vars = ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code']
net_melted = net_raw.melt(
    id_vars=id_vars,
    var_name='Year',
    value_name='Internet Users (% of Pop)'
)
net_2019 = net_melted[net_melted['Year'] == '2019'].copy()
net_2019 = net_2019.rename(columns={'Country Name': 'Country'})
net = net_2019[['Country', 'Internet Users (% of Pop)',]]
net

## Addressing the key mismatches
If we had merged the data above, we would have missing values in the final data frame even in rows where we have all the data.\
\
This is because of the mismatching naming of the countries and territories.\
For example, the hap data frame uses the label "Russia" for the 'gdp' label "Russian Federation".\
The left-merge algorithm would see these two labels as different rows, and wouldn't extend the relevant row.\
(credits for this observation and for its solution: Aslbek).\
\
In order to introduce more consistent naming, we identified all the mismatching territory names and made a new dictionary with corrections.\
World Bank's naming in the gdp data frame seemed more consistent, so we replaced some hap labels with gdp labels.\
\
Significantly missing blocks of data were extrapolated with a K-Nearest-Neighborhood algorithm (KNN).\
Shortly speaking, based on the existing data we "predicted" the missing values by taking an average of the K nearest neighbors.\
Neighborhood is defined by the closeness in other existing values.\
For example, we implemented KNN for Education expenditure column that lacked 23 rows.\
\
Finally, remaining missing values were filled by reasonably recent values.\
For example, for South Sudan "2019 GDP per capita" field was filled by "2015 GDP per capita".\
We also used external data in order to fill in minor missing values.

In [ ]:
print("Missmatch between happienss and GDP data sets: ", set(hap['Country']) - set(gdp['Country']), "\n")      # country names that 'hap' has, but 'gdp' doesn't
print("Missmatch between happiness and Education data sets: ", set(hap['Country']) - set(edu['Country']), "\n")
print("Missmatch between happiness and Life Expectancy data sets: ", set(hap['Country']) - set(exp['Country']), "\n")
print("Missmatch between happiness and InternetRates data sets: ", set(hap['Country']) - set(net['Country']), "\n")

In [ ]:
# ================================
# Correcting the labels
# ================================

# Values are the new, more universal names
# e.g. 'Kyrgyzstan' --> 'Kyrgyz Republic' (World Bank)

name_corrections = {
	'Congo (Brazzaville)': 'Congo, Dem. Rep.',
	'Congo (Kinshasa)': 'Congo, Rep.',
	'Czech Republic': 'Czechia',
	'Egypt': 'Egypt, Arab Rep.',
	'Gambia': 'Gambia, The',
	'Hong Kong': 'Hong Kong SAR, China',
	'Iran': 'Iran, Islamic Rep.',
	'Kyrgyzstan': 'Kyrgyz Republic',
	'Northern Cyprus': 'Northern Mariana Islands',
	'Palestinian Territories': 'West Bank and Gaza',
	'Russia': 'Russian Federation',
	'Slovakia': 'Slovak Republic',
	'Somalia': 'Somalia, Fed. Rep.',
	'South Korea': 'Korea, Rep.',
	'Syria': 'Syrian Arab Republic',
	'Trinidad & Tobago': 'Trinidad and Tobago',
	'Turkey': 'Turkiye',
	'Venezuela': 'Venezuela, RB',
	'Vietnam': 'Viet Nam',
	'Yemen': 'Yemen, Rep.',
	'Ivory Coast': 'Cote d\'Ivoire',
	'Laos': 'Lao PDR',
	'Swaziland': 'Eswatini',
}

for k, v in name_corrections.items():
    hap.loc[hap['Country'] == k, 'Country'] = v
    edu.loc[edu['Country'] == k, 'Country'] = v
    exp.loc[exp['Country'] == k, 'Country'] = v

print("Missmatch between happienss and GDP per capita data sets:")
#set(hap['Country']) - set(gdp['Country'])           # Any left out names

In [ ]:
# ================================
# Left merging. Raw data frame
# ================================
df_raw = (
    hap
    .merge(gdp, on=["Country"], how="left")
    .merge(edu, on=["Country"], how="left")
    .merge(exp, on=["Country"], how="left")
    .merge(net, on=["Country"], how="left")
)

print("Raw merged main data frame:")
df_raw

## Data Cleaning

In [ ]:
# ================================
# Addressing the missing data
# ================================
df_raw.isna().sum()

In [ ]:
# ================================
# Filling in the missing GDP per capita data 
# ================================
gdp_filling = {
    'Venezuela, RB': 15943.61,      # from 2014
    'Yemen, Rep.': 633.89,          # from 2018
    'South Sudan': 1080.15,         # from 2015
    'Taiwan': 36000                 # Find a source
}

for k, v in gdp_filling.items():
    df_raw.loc[df_raw['Country'] == k, 'GDP per capita'] = v

# ================================
# Filling in the missing Internet Propogation data 
# ================================
net_filling={
    # https://report.twnic.tw/2019/assets/download/TWNIC_TaiwanInternetReport_2019_EN.pdf
    "Taiwan":85.6,
    "Kosovo":89.443,                #from 2018
    "Northern Mariana Islands":66,
    "Libya":87.575,                 #from 2020
    "Tajikistan":27.544,            #from 2020
    "Turkmenistan":21.251,          #from 2017
    "Uganda":7.4,                   #from 2020
    "Yemen, Rep.":13.815,           #from 2020
    "Congo, Rep.":24.821,           #from 2020
    "Somalia, Fed. Rep.":15.024,    #from 2020
    "Venezuela, RB":61.6,           #from 2017
}

for k, v in net_filling.items():
    df_raw.loc[df_raw['Country'] == k, 'Internet Users (% of Pop)'] = v

# ================================
# Filling in the missing GDP per capita data 
# ================================
exp_filling = {
    # https://taiwantoday.tw/Society/Top-News/182736/Average-life-expectancy-hits-80.86-in-Taiwan
    "Taiwan": 80.86,
}

for k,v in exp_filling.items():
    df_raw.loc[df_raw["Country"]== k, "Life expectancy"] = v


In [ ]:
# ================================
# We used KNN model to extrapolate education expenditure based on neighbors in remaining features 
# ================================
features = df_raw.drop(columns=['Country'])

imputer = KNNImputer(n_neighbors=5, weights='distance')

imputed_arr = imputer.fit_transform(features)

imputed = pd.DataFrame(imputed_arr, columns=features.columns)

imputed['Country'] = df_raw['Country']

df_raw['Education expenditure'] = imputed['Education expenditure']


In [ ]:
# ================================
# Processed data
# ================================
df = df_raw.sort_values(by="Happiness score", ascending=False)

print("Merged main data frame (ranked by happiness score):")
df


In [ ]:
# ================================
# Checking for missing values
# ================================
print("Checking for missing values:")
print(df.isna().sum())


# Analysis
In the subsection 2.1, we plotted the main statistics.
In the subsection 2.2, we used regression models and MSE to evaluate their accuracy.


## General statistics

In [ ]:
# ================================
# Basic statistics
# ================================
print("Basic statistics:")
df.describe()


In [ ]:
# ================================
# Top 10 Countries by Happiness Score
# ================================
top_10 = df[["Country", "Happiness score"]].head(10) # df is already sorted
print("Top 10 happy countries:")
top_10


In [ ]:
# ================================
# Bottom 10 Countries by Happiness Score
# ================================
bottom_10 = df[["Country", "Happiness score"]].tail(10)
print("Bottom 10 happy countires:")
bottom_10


In [ ]:
# ================================
# Correlation matrix (Heat map)
# ================================
corr_matrix = df.select_dtypes(include=['number']).corr()   # Other ways to draw the heat map?
print("Correlation matrix:")
corr_matrix


In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(corr_matrix, annot=True, cmap='RdGy')
plt.show()


## Regression models

In [ ]:
# Constant data
x = df['GDP per capita']
y = df['Happiness score']

features = df.drop(columns = ['Country', 'Happiness score'])

In [ ]:
# ================================
# Scatter plot for a single feature (GDP per capita)
# ================================
plt.title('Happiness score vs. GDP per capita')
plt.xlabel('GDP per capita')
plt.ylabel('Happiness index')
plt.grid(True)
plt.scatter(x, y)

countries_to_label = {'Finland', 'China', 'Kazakhstan', 'United States', 'South Sudan', 'Mali'}

for country in countries_to_label:
    row = df[df['Country'] == country]
    plt.text(row['GDP per capita'], row['Happiness score'], country, weight='bold')


In [ ]:
# ================================
# Polynomial Regression. Correlation between GDP per capita and Happiness score
# ================================
# Keep x as a 1D Series for plotting and polyfit, but create a 2D feature array for sklearn
X = x.values.reshape(-1, 1)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def mse_evaluation(model, x_test, y_test):
    y_pred = model.predict(x_test)
    mse = mean_squared_error(y_test, y_pred)
    return mse

degree = [1, 2, 3, 4]
colors = ['red', 'orange', 'green', 'purple']

x_line = np.linspace(x.min(), x.max(), 100)

for deg, color in zip(degree, colors):
    coeffs = np.polyfit(x, y, deg=deg)
    poly = np.poly1d(coeffs)
    
    model = make_pipeline(
        PolynomialFeatures(degree=deg, include_bias=False),
        LinearRegression()
    )

    model.fit(x_train, y_train)
    mse = mse_evaluation(model, x_test, y_test)

    plt.figure()
    plt.grid(True)
    plt.scatter(x, y)
    
    plt.title(f'Regression of degree {deg}')
    plt.text(0.05, 0.95, f"MSE = {mse:.3f}", transform=plt.gca().transAxes, va='top')
    plt.plot(x_line, poly(x_line), color=color, label=f"Degree {deg}")
    
    plt.show()


### Logarithmic regression
We have noticed that the pattern is rather exponential (logarithmic), therefore we used logarithmic regression model by transforning the x axis logarithmically. Then we applied linear regression model and obtained coefficents , the intercept and MSE. We found out that it was surprisingly fitting well. (credits for this discovery: Olzhas)

In [ ]:
# Feature and target
X = df[['GDP per capita']]  # 2D DataFrame (single column)
y = df['Happiness score']   # 1D Series

# Only positive X (log requires >0) — use pandas indexing instead of numpy-style slicing
mask = X['GDP per capita'] > 0
X = X.loc[mask]
y = y.loc[mask]

# Log-transform X
X_log = np.log(X)

# Fit linear regression
model = LinearRegression()
model.fit(X_log, y)

# Predictions on actual data for MSE
y_pred_train = model.predict(X_log)
mse = mean_squared_error(y, y_pred_train)

print(f"Logarithmic model: MSE = {mse:.4f}")

# Smooth curve for plotting
x_min = X['GDP per capita'].min()
x_max = X['GDP per capita'].max()
x_line = np.linspace(x_min, x_max, 200).reshape(-1,1)
y_line = model.predict(np.log(x_line))

# Plot
plt.figure(figsize=(7,5))
plt.scatter(X['GDP per capita'], y, alpha=0.6, label='Data')
plt.plot(x_line.ravel(), y_line, color='red', label='Logarithmic fit')
plt.xlabel("GDP per capita")
plt.ylabel("Happiness score")
plt.title("Logarithmic Regression: GDP vs Happiness")
plt.text(0.05, 0.95, f"MSE = {mse:.3f}", transform=plt.gca().transAxes, va='top')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Feature and target
X = df['GDP per capita'].values.reshape(-1, 1)
y = df['Happiness score'].values

# Remove non-positive values (log undefined)
mask = X[:,0] > 0
X = X[mask]
y = y[mask]

# Log-transform X for modeling
X_log = np.log(X)

# Fit linear regression
model = LinearRegression()
model.fit(X_log, y)

# Predictions on actual data (for MSE)
y_pred = model.predict(X_log)
mse = mean_squared_error(y, y_pred)
print(f"Logarithmic regression MSE: {mse:.4f}")

# Smooth curve for plotting
x_line = np.linspace(X.min(), X.max(), 200).reshape(-1,1)
y_line = model.predict(np.log(x_line))

# Plot scatter + regression curve
plt.figure(figsize=(7,5))
plt.scatter(X, y, alpha=0.6, label='Data')
plt.plot(x_line, y_line, color='red', label='Logarithmic fit')

# Log-scale X-axis for visual linearization
plt.xscale('log')

# Display MSE on plot
plt.text(0.05, 0.95, f"MSE = {mse:.3f}", transform=plt.gca().transAxes, va='top')

plt.xlabel("GDP per capita (log scale)")
plt.ylabel("Happiness score")
plt.title("Logarithmic Regression: GDP vs Happiness")
plt.grid(True, which='both', ls='--')
plt.legend()
plt.show()

## Feature correlation
We examined various relationships between features and the happiness index.
There are multiple scatter plots with linear regression line and compared their correlation matirx. 
The results were sorted and are as follows:
    * The most significant correlation:
    * Less significant coorelation:
    * Weak correlation:

One can note that there are no negative correlation even amng insignificant ones. This shows that the feature choice is quite lucky.

Next, is a general linear regression snippet that captures the relationships between all the features. We considered it noteworkthy to write down it r^2 and MSE evaluation scores. 

In [ ]:
for col_name in features.drop(columns = ['GDP per capita']).columns:
    X_col = df[[col_name]].values
    y = df['Happiness score'].values
    
    # Fit linear model
    model = LinearRegression()
    model.fit(X_col, y)
    
    # MSE on training data
    y_pred = model.predict(X_col)
    mse = mean_squared_error(y, y_pred)
    print(f"{col_name}: MSE = {mse:.3f}")
    
    # Scatter plot + regression line
    plt.figure()
    plt.scatter(X_col, y, alpha=0.6)
    x_line = np.linspace(X_col.min(), X_col.max(), 200).reshape(-1,1)
    y_line = model.predict(x_line)
    plt.plot(x_line, y_line, color='red', label='Linear fit')
    
    plt.title(f"Happiness vs {col_name}")
    plt.xlabel(col_name)
    plt.ylabel("Happiness score")
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# ================================
# Linear Regression Model for Multiple Features
# ================================
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.2, random_state=42)

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("R^2 score: ", r2)
print("Mean Squared Error: ", mse)

coefficients = pd.DataFrame({
    "Feature": features.columns,
    "Coefficient": lr.coef_
}).sort_values("Coefficient", ascending=False)

print("\nFeatures' Importance (Linear Regression Coefficients):")
print(coefficients)

plt.figure(figsize=(10,6))
plt.bar(coefficients["Feature"], coefficients["Coefficient"], color='skyblue')
plt.xticks(rotation=45, ha='right')
plt.xlabel("Feature")
plt.ylabel("Coefficient Value")
plt.title("Feature Importance (Linear Regression Coefficients)")
plt.tight_layout()
plt.show()


## Clusters

We used KMeans as clustering model. (Task maybe try others?)
Our model has divided the model into three clusters.
We used GDP per capita vs. Happiness scores to illustrate those clusters, although they used more features.

Task: what are those models. Describe them
Task: Put the centroids.



In [ ]:
# ================================
# Clustering
# ================================

scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X_scaled)

df_cluster = df.copy()
df_cluster['Cluster'] = clusters

plt.scatter(df_cluster['GDP per capita'], df_cluster['Happiness score'], c = df_cluster['Cluster'], cmap='Set1')
plt.xlabel('GDP per capita')
plt.ylabel('Happiness score')
plt.title('Country clusters')
plt.colorbar(label='Cluster')
plt.show


for country in countries_to_label:
    row = df[df['Country'] == country]
    plt.text(row['GDP per capita'].values[0], row['Happiness score'].values[0], country, fontsize=9, weight='bold')



In [ ]:
for col in features.drop(columns = ['GDP per capita']).columns:

    plt.figure()
    plt.scatter(df_cluster[col], df_cluster['Happiness score'], c = df_cluster['Cluster'], cmap='Set1')
    plt.xlabel(f"{col}")
    plt.ylabel('Happiness score')
    plt.colorbar(label='Cluster')
    
    
    for country in countries_to_label:
        row = df[df['Country'] == country]
        plt.text(row[col].values[0], row['Happiness score'].values[0], country, fontsize=9, weight='bold')
    


In [ ]:
clusters_dict = {}

for c in sorted(df_cluster['Cluster'].unique()):
    clusters_dict[c] = df_cluster[df_cluster['Cluster'] == c]['Country'].tolist()

for c, countries in clusters_dict.items():
    print(f"Cluster {c}:")
    print(countries)
    print("\n")

'''
cluster_summary = df_cluster.groupby('Cluster')
len([features.columns])
print(cluster_summary)
'''

centroids = pd.DataFrame(scaler.inverse_transform(kmeans.cluster_centers_), columns=features.columns)
centroids

# Evaluation of the Report

There are limitations: 
only 2019, 
incomplete data (Education expenditure, GDP per capita), 
extrapolations when handlind missing values, 
only three data sets


This study could extended. 
Other years and global trends, for example comparing with pandemic times.
Other features such as climate, ...

# Summary

We have identified ...

# Dashboard
The interactive dashboard below perfectly summarizes our analysis. 
The reader can hover over regions and see their happiness scores.
The lighter the color, the happier the region is.
(credits for drawing the map: Yernur)

In [ ]:
fig = px.choropleth(hap[["Country", "Happiness score"]], 
                    locations='Country', 
                    color='Happiness score',
                    locationmode='country names', 
                    color_continuous_scale=px.colors.sequential.speed_r,
                    title='Global Happiness Score (2019)',
                    height=720
                   )

fig.update_xaxes(automargin=True)
fig.update_yaxes(automargin=True)

http://localhost:8051

In [ ]:
df.to_csv("final_data.csv")